In [1]:
#using K-Means clustering to build a product recommendation system from two different datasets
#one dataset has movies and the other has ratings made by users that have watched the movie; common field is movie_id


In [1]:
import zipfile
with zipfile.ZipFile('C:\Input\ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [4]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

# Count the number of unique users and items
n_users = ratings_df['userId'].nunique()
n_items = ratings_df['movieId'].nunique()

In [3]:
# Data Citation:
# F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on 
# Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>

! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  2  955k    2 27195    0     0  13880      0  0:01:10  0:00:01  0:01:09 13889
 43  955k   43  412k    0     0   121k      0  0:00:07  0:00:03  0:00:04  121k
100  955k  100  955k    0     0   268k      0  0:00:03  0:00:03 --:--:--  268k


In [5]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [6]:
# Take a look at movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")

Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [9]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)
        
    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)
    
    def predict(self, user, item):
        return self.forward(user, item)

In [10]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()
        
        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()
        
        #--- Producing new continuous IDs for users and movies ---
        
        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}
        
        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}
        
        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])
        
        
        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [11]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: False
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0421, 0.0249, 0.0477,  ..., 0.0141, 0.0013, 0.0341],
        [0.0150, 0.0001, 0.0115,  ..., 0.0284, 0.0046, 0.0274],
        [0.0307, 0.0013, 0.0055,  ..., 0.0382, 0.0046, 0.0376],
        ...,
        [0.0309, 0.0116, 0.0189,  ..., 0.0156, 0.0042, 0.0494],
        [0.0217, 0.0350, 0.0418,  ..., 0.0278, 0.0267, 0.0410],
        [0.0276, 0.0088, 0.0242,  ..., 0.0196, 0.0349, 0.0440]])
item_factors.weight tensor([[0.0250, 0.0248, 0.0211,  ..., 0.0226, 0.0194, 0.0394],
        [0.0411, 0.0169, 0.0333,  ..., 0.0386, 0.0356, 0.0331],
        [0.0114, 0.0432, 0.0267,  ..., 0.0192, 0.0009, 0.0035],
        ...,
        [0.0148, 0.0252, 0.0274,  ..., 0.0242, 0.0080, 0.0353],
        [0.0104, 0.0191, 0.0493,  ..., 0.0264, 0.0261, 0.0491],
        [0.0429, 0.0265, 0.0457,  ..., 0.0281, 0.0132, 0.0314]])


In [12]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
         if cuda:
            x, y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
            print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

<ipython-input-12-d512f2c473f0>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for it in tqdm(range(num_epochs)):


  0%|          | 0/128 [00:00<?, ?it/s]

In [13]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0 
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
            uw = param.data
            c +=1
        else:
            iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[0.0421, 0.0249, 0.0477,  ..., 0.0141, 0.0013, 0.0341],
        [0.0150, 0.0001, 0.0115,  ..., 0.0284, 0.0046, 0.0274],
        [0.0307, 0.0013, 0.0055,  ..., 0.0382, 0.0046, 0.0376],
        ...,
        [0.0309, 0.0116, 0.0189,  ..., 0.0156, 0.0042, 0.0494],
        [0.0217, 0.0350, 0.0418,  ..., 0.0278, 0.0267, 0.0410],
        [0.0276, 0.0088, 0.0242,  ..., 0.0196, 0.0349, 0.0440]])
item_factors.weight tensor([[0.0250, 0.0248, 0.0211,  ..., 0.0226, 0.0194, 0.0394],
        [0.0411, 0.0169, 0.0333,  ..., 0.0386, 0.0356, 0.0331],
        [0.0114, 0.0432, 0.0267,  ..., 0.0192, 0.0009, 0.0035],
        ...,
        [0.0148, 0.0252, 0.0274,  ..., 0.0242, 0.0080, 0.0353],
        [0.0104, 0.0191, 0.0493,  ..., 0.0264, 0.0261, 0.0491],
        [0.0429, 0.0265, 0.0457,  ..., 0.0281, 0.0132, 0.0314]])


In [14]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [15]:
len(trained_movie_embeddings) # unique movie factor weights

9724

In [16]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

In [18]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
    print("Cluster #{}".format(cluster))
    movs = []
    for movidx in np.where(kmeans.labels_ == cluster)[0]:
        movid = train_set.idx2movieid[movidx]
        rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
        movs.append((movie_names[movid], rat_count))
    for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]: 
        print("\t", mov[0])

Cluster #0
	 Apollo 13 (1995)
	 True Lies (1994)
	 Back to the Future (1985)
	 Speed (1994)
	 Star Wars: Episode I - The Phantom Menace (1999)
	 Monty Python and the Holy Grail (1975)
	 Reservoir Dogs (1992)
	 Babe (1995)
	 Truman Show, The (1998)
	 Twister (1996)
Cluster #1
	 Shawshank Redemption, The (1994)
	 Pulp Fiction (1994)
	 Jurassic Park (1993)
	 Dark Knight, The (2008)
	 Pirates of the Caribbean: The Curse of the Black Pearl (2003)
	 Mrs. Doubtfire (1993)
	 Catch Me If You Can (2002)
	 Clear and Present Danger (1994)
	 Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
	 Trainspotting (1996)
Cluster #2
	 Forrest Gump (1994)
	 Star Wars: Episode IV - A New Hope (1977)
	 Independence Day (a.k.a. ID4) (1996)
	 Star Wars: Episode VI - Return of the Jedi (1983)
	 Sixth Sense, The (1999)
	 Lion King, The (1994)
	 Alien (1979)
	 Aliens (1986)
	 Willy Wonka & the Chocolate Factory (1971)
	 Bourne Identity, The (2002)
Cluster #3
	 Fugitive, 

In [ ]:
'''In a nut shell, this code demonstrates how to use unsupervised machine learning techniques to 
cluster movies based on their ratings. 
Specifically, it uses the KMeans algorithm to cluster movies into 10 clusters based on the similarity of their ratings. 
The purpose of this code is to group movies into similar categories, which could be used for various purposes such as 
recommending similar movies to users or for content categorization.

Here's a step-by-step breakdown of what the code does:

It imports necessary libraries and loads the movie lens dataset.
It creates a ratings matrix using the dataset, where rows correspond to users and columns correspond to movies. 
Each element of the matrix represents the rating given by a user to a movie.
It performs dimensionality reduction on the matrix using Singular Value Decomposition (SVD) 
to reduce the number of features (i.e., movies) and improve the efficiency of clustering.
It uses the KMeans algorithm to cluster the movies into 10 groups based on the similarity of their ratings.
For each cluster, it prints out the top 10 movies with the highest number of ratings.
I hope this helps!'''